In [1]:
import numpy as np
import tensorflow as tf
import gym
env=gym.make('CartPole-v0')

[2017-06-07 15:29:23,994] Making new env: CartPole-v0


In [2]:
env.reset()
random_episodes=0
reward_sum=0
while random_episodes<10:
#     env.render()
    observation,reward,done,_=env.step(np.random.randint(0,2))
    reward_sum+=reward
    if done:
        random_episodes+=1
        print("Reward for this episode was:",reward_sum)
        reward_sum=0
        env.reset()

Reward for this episode was: 9.0
Reward for this episode was: 11.0
Reward for this episode was: 17.0
Reward for this episode was: 16.0
Reward for this episode was: 18.0
Reward for this episode was: 33.0
Reward for this episode was: 51.0
Reward for this episode was: 32.0
Reward for this episode was: 22.0
Reward for this episode was: 15.0


In [3]:
H=50
batch_size=25
learning_rate=0.15
D=4
gamma=0.99


In [4]:
observations=tf.placeholder(tf.float32,[None,D],name="input_x")
w1=tf.get_variable("w1",shape=[D,H],initializer=tf.contrib.layers.xavier_initializer())
layer1=tf.nn.relu(tf.matmul(observations,w1))
w2=tf.get_variable("w2",shape=[H,1],initializer=tf.contrib.layers.xavier_initializer())
score=tf.matmul(layer1,w2)
probability=tf.nn.sigmoid(score)

In [5]:
adam=tf.train.AdamOptimizer(learning_rate=learning_rate)
w1Grad=tf.placeholder(tf.float32,name="batch_grad1")
w2Grad=tf.placeholder(tf.float32,name="batch_grad2")
batchGrad=[w1Grad,w2Grad]

tvars=tf.trainable_variables()
updateGrads=adam.apply_gradients(zip(batchGrad,tvars))


In [6]:
def discount_rewards(r):
    discounted_r = np.zeros_like(r)
    running_add = 0
    for t in reversed(range(r.size)):
        running_add=running_add*gamma+r[t]
        discounted_r[t]=running_add
        
    return discounted_r

In [7]:
input_y = tf.placeholder(tf.float32,[None,1],name="input_y")
advantages = tf.placeholder(tf.float32,name="reward_signal")
loglik=tf.log(input_y*(input_y-probability)+(1-input_y)*(input_y+probability))
loss = -tf.reduce_mean(loglik*advantages)


newGrads=tf.gradients(loss,tvars)

In [8]:
xs,ys,drs=[],[],[]
reward_sum = 0
episode_number = 1
total_episodes=10000

In [9]:
with tf.Session() as sess:
    rendering=False
    init=tf.global_variables_initializer()
    sess.run(init)
    observation=env.reset()
    
    gradBuffer=sess.run(tvars)
    for ix,grad in enumerate(gradBuffer):
        gradBuffer[ix] = grad*0
    control=True    
    while episode_number <= total_episodes:
        if reward_sum/batch_size > 190 or rendering == True:
            env.render()
            rendering=True
            
        x = np.reshape(observation,[1,D])
        
        tfprob = sess.run(probability,feed_dict={observations:x})
        if control:
            control=False
            throttle=0.9
        else:
            control=True
            throttle=0.1
            
        action  = 1 if  tfprob > throttle else 0
        
        xs.append(x)
        y=1 - action
        ys.append(y)
        
        observation,reward,done,infor=env.step(action)
        reward_sum +=reward
        drs.append(reward)
        
        if done:
            episode_number+=1
            epx = np.vstack(xs)
            epy = np.vstack(ys)
            epr = np.vstack(drs)
            xs,ys,drs=[],[],[]
            
            discounted_epr= discount_rewards(epr)
            discounted_epr -= np.mean(discounted_epr)
            discounted_epr /= np.std(discounted_epr)
            
            tGrad = sess.run(newGrads,feed_dict={observations:epx,
                                                 input_y:epy,advantages:discounted_epr})
#             print("advantages:",tf.shape(discounted_epr).eval())
            for ix,grad in enumerate(tGrad):
                gradBuffer[ix] +=grad
            
            if episode_number % batch_size == 0:
                sess.run(updateGrads,feed_dict = {w1Grad:gradBuffer[0]
                                                  ,w2Grad:gradBuffer[1]})
                
#                 print('w1Grag mean',gradBuffer[0],
#                       'w2Grag mean',(tf.reduce_mean(gradBuffer[1])).eval()) 
                
                for ix,grad in enumerate(gradBuffer):
                    gradBuffer[ix] = grad*0
                    
                print('Average reward for episode %d : %f' %  
                     (episode_number,reward_sum/batch_size))
                
                print('epx length %d ' %  len(epx) )   
#                 print('w1Grag shape',(tf.shape(gradBuffer[0])).eval(),
#                       'w2Grag shape',(tf.shape(gradBuffer[1])).eval())              
                
                if reward_sum/batch_size > 180:
                    batch_size=100
            
                if reward_sum/batch_size > 195:
                    print("Task solved in",episode_number,'episodes!')
                    break
                    
                reward_sum =0
                
            observation = env.reset()

Average reward for episode 25 : 34.600000
epx length 25 
Average reward for episode 50 : 35.560000
epx length 30 
Average reward for episode 75 : 37.080000
epx length 56 
Average reward for episode 100 : 44.320000
epx length 52 
Average reward for episode 125 : 57.120000
epx length 46 
Average reward for episode 150 : 88.640000
epx length 43 
Average reward for episode 175 : 166.520000
epx length 150 
Average reward for episode 200 : 180.680000
epx length 200 
Average reward for episode 300 : 192.700000
epx length 200 
Average reward for episode 400 : 197.640000
epx length 200 
Task solved in 400 episodes!
